In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [3]:
def standardize(train, test):
    mean, std = train.mean(), train.std()
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test

In [4]:
x_train, x_test = standardize(x_train, x_test)



In [5]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['LeakyReLU']

In [7]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(alpha=1.5),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=1.0) 
        }

        model_name = 'std_' + act + '_1.5_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation LeakyReLU with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.5894 - acc: 0.8386 - val_loss: 0.3429 - val_acc: 0.9031
Epoch 2/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.4068 - acc: 0.8854 - val_loss: 0.3196 - val_acc: 0.9046
Epoch 3/100
60000/60000 [==============================] - 2s 40us/step - loss: 0.3637 - acc: 0.8974 - val_loss: 0.2850 - val_acc: 0.9163
Epoch 4/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.3415 - acc: 0.9032 - val_loss: 0.2803 - val_acc: 0.9219
Epoch 5/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.3270 - acc: 0.9080 - val_loss: 0.2794 - val_acc: 0.9218
Epoch 6/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.3176 - acc: 0.9093 - val_loss: 0.2572 - val_acc: 0.9285
Epoch 7/100
60000/60000 [==============================] - 3s 58u

Epoch 59/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.2553 - acc: 0.9281 - val_loss: 0.2308 - val_acc: 0.9366
Epoch 60/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2547 - acc: 0.9259 - val_loss: 0.2160 - val_acc: 0.9404
Epoch 61/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2545 - acc: 0.9276 - val_loss: 0.2284 - val_acc: 0.9367
Epoch 62/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.2535 - acc: 0.9273 - val_loss: 0.2342 - val_acc: 0.9379
Epoch 63/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.2540 - acc: 0.9283 - val_loss: 0.2171 - val_acc: 0.9392
Epoch 64/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2530 - acc: 0.9283 - val_loss: 0.2485 - val_acc: 0.9321
Epoch 65/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.2553 - acc: 0.9277 - val_loss: 0.2219 - val_acc: 0.9393
Epoch 66/100
60000/60000 [=

Epoch 17/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2785 - acc: 0.9206 - val_loss: 0.2375 - val_acc: 0.9328
Epoch 18/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2755 - acc: 0.9210 - val_loss: 0.2451 - val_acc: 0.9287
Epoch 19/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2743 - acc: 0.9214 - val_loss: 0.2249 - val_acc: 0.9390
Epoch 20/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2749 - acc: 0.9213 - val_loss: 0.2355 - val_acc: 0.9335
Epoch 21/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2726 - acc: 0.9227 - val_loss: 0.2459 - val_acc: 0.9324
Epoch 22/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2709 - acc: 0.9229 - val_loss: 0.2455 - val_acc: 0.9309
Epoch 23/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2690 - acc: 0.9239 - val_loss: 0.2440 - val_acc: 0.9312
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2552 - acc: 0.9274 - val_loss: 0.2306 - val_acc: 0.9373
Epoch 77/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2519 - acc: 0.9297 - val_loss: 0.2203 - val_acc: 0.9413
Epoch 78/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2521 - acc: 0.9282 - val_loss: 0.2189 - val_acc: 0.9421
Epoch 79/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2486 - acc: 0.9278 - val_loss: 0.2260 - val_acc: 0.9359
Epoch 80/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2520 - acc: 0.9280 - val_loss: 0.2270 - val_acc: 0.9363
Epoch 81/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2531 - acc: 0.9282 - val_loss: 0.2271 - val_acc: 0.9387
Epoch 82/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2495 - acc: 0.9285 - val_loss: 0.2153 - val_acc: 0.9395
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2619 - acc: 0.9254 - val_loss: 0.2322 - val_acc: 0.9357
Epoch 35/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2617 - acc: 0.9256 - val_loss: 0.2241 - val_acc: 0.9387
Epoch 36/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2661 - acc: 0.9237 - val_loss: 0.2269 - val_acc: 0.9360
Epoch 37/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2598 - acc: 0.9261 - val_loss: 0.2274 - val_acc: 0.9356
Epoch 38/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2598 - acc: 0.9260 - val_loss: 0.2393 - val_acc: 0.9311
Epoch 39/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2627 - acc: 0.9268 - val_loss: 0.2214 - val_acc: 0.9403
Epoch 40/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2570 - acc: 0.9269 - val_loss: 0.2366 - val_acc: 0.9361
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2502 - acc: 0.9285 - val_loss: 0.2186 - val_acc: 0.9404
Epoch 94/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2533 - acc: 0.9278 - val_loss: 0.2278 - val_acc: 0.9381
Epoch 95/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.2499 - acc: 0.9291 - val_loss: 0.2293 - val_acc: 0.9343
Epoch 96/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.2515 - acc: 0.9286 - val_loss: 0.2269 - val_acc: 0.9391
Epoch 97/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2487 - acc: 0.9281 - val_loss: 0.2292 - val_acc: 0.9386
Epoch 98/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2480 - acc: 0.9302 - val_loss: 0.2367 - val_acc: 0.9359
Epoch 99/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2489 - acc: 0.9289 - val_loss: 0.2405 - val_acc: 0.9331
Epoch 100/100
60000/60000 [

60000/60000 [==============================] - 3s 47us/step - loss: 0.2566 - acc: 0.9280 - val_loss: 0.2204 - val_acc: 0.9383
Epoch 51/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.2566 - acc: 0.9274 - val_loss: 0.2334 - val_acc: 0.9368
Epoch 52/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2563 - acc: 0.9271 - val_loss: 0.2376 - val_acc: 0.9362
Epoch 53/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.2550 - acc: 0.9268 - val_loss: 0.2322 - val_acc: 0.9357
Epoch 54/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.2544 - acc: 0.9281 - val_loss: 0.2401 - val_acc: 0.9332
Epoch 55/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.2572 - acc: 0.9268 - val_loss: 0.2265 - val_acc: 0.9377
Epoch 56/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.2571 - acc: 0.9254 - val_loss: 0.2322 - val_acc: 0.9357
Epoch 57/100
60000/60000 [==============

Epoch 8/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.3052 - acc: 0.9128 - val_loss: 0.2432 - val_acc: 0.9322
Epoch 9/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2975 - acc: 0.9157 - val_loss: 0.2443 - val_acc: 0.9304
Epoch 10/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2946 - acc: 0.9155 - val_loss: 0.2545 - val_acc: 0.9286
Epoch 11/100
60000/60000 [==============================] - 3s 58us/step - loss: 0.2925 - acc: 0.9159 - val_loss: 0.2650 - val_acc: 0.9261
Epoch 12/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.2902 - acc: 0.9169 - val_loss: 0.2455 - val_acc: 0.9314
Epoch 13/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.2864 - acc: 0.9186 - val_loss: 0.2400 - val_acc: 0.9344
Epoch 14/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.2854 - acc: 0.9190 - val_loss: 0.2511 - val_acc: 0.9308
Epoch 15/100
60000/60000 [===

60000/60000 [==============================] - 3s 56us/step - loss: 0.2567 - acc: 0.9265 - val_loss: 0.2202 - val_acc: 0.9382
Epoch 67/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.2548 - acc: 0.9266 - val_loss: 0.2309 - val_acc: 0.9344
Epoch 68/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.2574 - acc: 0.9267 - val_loss: 0.2329 - val_acc: 0.9352
Epoch 69/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.2560 - acc: 0.9279 - val_loss: 0.2276 - val_acc: 0.9392 acc: 
Epoch 70/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.2577 - acc: 0.9275 - val_loss: 0.2334 - val_acc: 0.9350
Epoch 71/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.2521 - acc: 0.9283 - val_loss: 0.2280 - val_acc: 0.9374
Epoch 72/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.2566 - acc: 0.9289 - val_loss: 0.2234 - val_acc: 0.9377
Epoch 73/100
60000/60000 [========